In [ ]:
import cv2
import numpy as np
import os


MIN_MATCH_COUNT=15



detector = cv2.SIFT_create()

bf=cv2.BFMatcher()
FLANN_INDEX_KDITREE=0
flannParam=dict(algorithm=FLANN_INDEX_KDITREE,tree=5)
flann=cv2.FlannBasedMatcher(flannParam,{})

trainImg=cv2.imread(r"C:\Users\abhir\OneDrive\Desktop\SIFT_TRAINING\webcametraining2.jpg",0)
trainKP,trainDesc=detector.detectAndCompute(trainImg,None)
##face1=cv2.imread(r"C:\Users\abhir\OneDrive\Desktop\webcametraining1.jpg",0)
#face2=cv2.imread(r"C:\Users\abhir\OneDrive\Desktop\cubetrainingimage2.jpg",0)
#face3=cv2.imread(r"C:\Users\abhir\OneDrive\Desktop\cubetrainingimage3.jpg",0)
#face4=cv2.imread(r"C:\Users\abhir\OneDrive\Desktop\cubetrainingimage4.jpg",0)


#face1KeyPoints,face1Desc=detector.detectAndCompute(face1,None)
#face2KeyPoints,face2Desc=detector.detectAndCompute(face2,None)
#face3KeyPoints,face3Desc=detector.detectAndCompute(face3,None)
#face4KeyPoints,face4Desc=detector.detectAndCompute(face4,None)##



cam=cv2.VideoCapture(0)
while True:
    ret, QueryImgBGR=cam.read()
    QueryImg=cv2.cvtColor(QueryImgBGR,cv2.COLOR_BGR2GRAY)
    queryKP,queryDesc=detector.detectAndCompute(QueryImg,None)
    matches=bf.knnMatch(queryDesc,trainDesc,k=2)

    goodMatch=[]
    for m,n in matches:
        if(m.distance<0.75*n.distance):
            goodMatch.append(m)
    if(len(goodMatch)>MIN_MATCH_COUNT):
        tp=[]
        qp=[]
        for m in goodMatch:
            tp.append(trainKP[m.trainIdx].pt)
            qp.append(queryKP[m.queryIdx].pt)
        tp,qp=np.float32((tp,qp))
        H,status=cv2.findHomography(tp,qp,cv2.RANSAC,3.0)
        h,w=trainImg.shape
        trainBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
        queryBorder=cv2.perspectiveTransform(trainBorder,H)
        cv2.polylines(QueryImgBGR,[np.int32(queryBorder)],True,(0,255,0),5)
    else:
        print ("Not Enough match found")
    cv2.imshow('result',QueryImgBGR)
    if cv2.waitKey(10)==ord('q'):
        break
cam.release()
cv2.destroyAllWindows()